# Q&A Chian Using API

In [ ]:
!pip install groq
!pip install -q langchain==0.0.219
!pip install -U cohere
!pip install -q chromadb==0.3.26


!pip install -q pandas==2.0.0
!pip install -q pypdf==3.12.0
!pip install -q unstructured==0.7.12
!pip install pydantic==1.10.9

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Embeddings | Cohere

In [ ]:
# Edit Class to work with cohere v3
    def embed_documents(self, texts: List[str], input_type: str  = 'search_document') -> List[List[float]]:
        """Call out to Cohere's embedding endpoint.

        Args:
            texts: The list of texts to embed.
            input_type: The type of input for the embedding.

        Returns:
            List of embeddings, one for each text.
        """
        embeddings = self.client.embed(
            model=self.model, texts=texts, truncate=self.truncate, input_type=input_type
        ).embeddings
        return [list(map(float, e)) for e in embeddings]


    def embed_query(self, text: str, input_type: str = 'search_query') -> List[float]:
        """Call out to Cohere's embedding endpoint.

        Args:
            text: The text to embed.

        Returns:
            Embeddings for the text.
        """
        embedding = self.client.embed(
            model=self.model, texts=[text], truncate=self.truncate, input_type=input_type
        ).embeddings[0]
        return list(map(float, embedding))

In [1]:
from google.colab import userdata

cohere_api_key = userdata.get('COHERE_API_KEY')

In [2]:
from langchain.embeddings  import CohereEmbeddings

embedding_llm = CohereEmbeddings(model="embed-multilingual-v3.0", cohere_api_key=cohere_api_key)

In [3]:
embedding_llm.embed_documents

<bound method CohereEmbeddings.embed_documents of CohereEmbeddings(client=<cohere.client.Client object at 0x7a9c02ab7730>, model='embed-multilingual-v3.0', truncate=None, cohere_api_key='BhHKBBDIHoDtCy2zf1V0rdYEfm4VPXq8RgDwIWp9')>

In [4]:
# query
query_text = "ما هو دور مسؤول لجنه الميديا؟"
query_embedding = embedding_llm.embed_query(query_text)
len(query_embedding)

1024

# Load VD

In [5]:
from langchain.vectorstores import Chroma

## Load from disk
load_from_dir = "/content/drive/MyDrive/Ma3an_QA/VD"

vector_db = Chroma(
    persist_directory=load_from_dir,
    embedding_function=embedding_llm
)

In [6]:
# search for most similar document to a query
query_text = "ما هو دور ميديا"

similar_docs = vector_db.similarity_search(
                                      query_text,
                                      k=5,
                                      filter={'source': 'لائحة'}
                                    )
similar_docs

[Document(page_content=')5(إﻧﻬﺎءاﻟﺘﺼﺎرﻳﺢواﻹﺟﺮاءاتاﻟﺨﺎﺻﺔﺑﻔﻌﺎﻟﻴﺎتاﻟﻔﺮع.\n\n●ﻟﺠﻨﺔاﻟﻨﺸﺎطActivity:\n)1(ﻣﺴﺌﻮﻟﺔﻋﻦﻛﺎﻓﺔأﻧﻮاعاﻷﻧﺸﻄﺔاﻟﺘﻰﺗﻘﺪﻣﻬﺎﻣﻌﺎ.)2(ﺗﻜﻮنﺿﻤﻦﻣﺴﺌﻮﻟﻴﺘﻬﺎﻋﻤﻞاﺳﺘﻄﻼعرأي)ﺑﻤﺴﺎﻋﺪةاﻟﻌﻼﻗﺎتاﻟﻌﺎﻣﺔ(.)3(وﺿﻊﺧﻄﺔﻋﺎﻣﺔﻟﻺﻳﭭﻨﺘﺎتﻃﻮالاﻟﻌﺎموﺗﺤﺪﻳﺪاﻟﻤﺤﺘﻮىواﻷﻫﺪافاﻟﻌﺎﻣﺔ.)4(اﻟﺒﺤﺚواﻟﺘﻄﻮﻳﺮﻓﻲاﻹﻳﭭﻨﺘﺎتﺷﻜﻠًﺎوﻣﻀﻤﻮﻧًﺎ.\n\n●ﻟﺠﻨﺔاﻟﻤﻴﺪﻳﺎ:\n)1(ﻣﺴﺌﻮﻟﺔﻋﻦاﻟﺪﻋﺎﻳﺔﻟﻠﻔﻌﺎﻟﻴﺎت.)2(اﻟﺘﺴﻮﻳﻖﻷﻓﻜﺎرﻣﻌﺎ.)3(ﺗﻮﺛﻴﻖأﻧﺸﻄﺔوﻓﻌﺎﻟﻴﺎتاﻷﺳﺮة.)4(ﺗﺘﻜﻮنﻣﻦ٤ﻓﺮقﻋﻤﻞ:)ﻓﻴﺪﻳﻮ،ﺗﺼﻤﻴﻢ،ﻫﺎﻧﺪ،ﺳﻮﺷﻴﺎل(.\n\n5|Page', metadata={'source': 'لائحة'}),
 Document(page_content=')2(إدارةاﻻﺟﺘﻤﺎﻋﺎتاﻟﺪورﻳﺔﺑﻴﻦﻣﺴﺌﻮﻟﻲاﻟﻤﺎﻟﻴﺔFund،وﻣﺘﺎﺑﻌﺔﻣﻬﺎماﻟﻠﺠﻨﺔاﻟﻤﺘﻌﻠﻘﺔﺑﻤﻌﺎاﻟﺠﺎﻣﻌﺔ.)3(ﻣﺴﺌﻮلﻋﻦاﻟﺘﻨﺴﻴﻖﺑﻴﻦﻟﺠﺎناﻟﻌﻼﻗﺎتاﻟﻌﺎﻣﺔﻟﻠﻔﺮوعﻟﻮﺿﻊاﻟﻤﺤﺘﻮىواﻟﻤﻌﺎﻳﻴﺮاﻟﻮاﺿﺤﺔﻟﻬﺎﺑﻴﻦاﻟﻔﺮوعوﻣﺘﺎﺑﻌﺔﺟﻮدةاﻟﺘﻨﻔﻴﺬ.\n\n)4(إدارةاﻻﺟﺘﻤﺎﻋﺎتاﻟﺪورﻳﺔﺑﻴﻦﻣﺴﺆوﻟﻲاﻟﻌﻼﻗﺎتاﻟﻌﺎﻣﺔ،وﻣﺘﺎﺑﻌﺔﻣﻬﺎماﻟﻠﺠﻨﺔاﻟﻤﺘﻌﻠﻘﺔﺑﻤﻌﺎاﻟﺠﺎﻣﻌﺔ.\n\n4|Page', metadata={'source': 'لائحة'}),
 Document(page_content='●ﻣﺴﺌﻮلاﻟﺒﺤﺚواﻟﺘﻄﻮﻳﺮ)R&D:(\n)1(ﻣﺴﺌﻮلﻋﻦوﺿﻊﻧﻬﺞﻣﺘﻘﺎربﻟﻠﺘﻄﻮﻳﺮﻓﻲﺟﻤﻴﻊاﻟﻔﺮوعﻣﻊﻣﺴﺌﻮﻟﻲاﻟﻤﻮارداﻟﺒﺸﺮﻳﺔأوﻣﺴﺌﻮﻟﻲﺗﻄﻮﻳﺮاﻟﻔﺮوعﺑﻤﺎﻳﺘﻨﺎﺳﺐﻣﻊوﺿﻊﻛﻞﻓﺮع،وﻣﺘﺎﺑﻌﺘﻪوﺗﺤﺴﻴﻨﻪورﻓﻊﺗﻘﺎرﻳﺮﻟﻠﻤﻨﺴﻖاﻟﻌﺎمﺑﻤﺎﺗﻢ

# Inference

In [7]:
def query_data(query):

    similar_docs = vector_db.similarity_search(
                                          query,
                                          k=5,
                                          # filter={'source': 'لائحة'}
                                        )
    context = str(similar_docs)


    contentMsg_template = "\n".join([
    "أجب عن السؤال التالي باستخدام السياق المُقدم.",
    "وقم بتنسيق الاجابة",
    "وعدم تكرار الاجابة",
    "إذا لم تكن الإجابة موجودة في السياق، قل 'لا تتوفر إجابة'",
    "### السياق:",
    "{context}",
    "",
    "### السؤال:",
    "{question}",
    "",
    "### الإجابة:"
    ,
    ])

    contentMsg = contentMsg_template.format(context=context, question=query)


    from groq import Groq
    import os
    from google.colab import userdata


    client = Groq(api_key = userdata.get('GROQ_API'))
    completion = client.chat.completions.create(
        model="llama-3.1-70b-versatile",
        messages=[
                  {
              "role": "user",
              "content": contentMsg,
          }
        ],
        temperature=0.3,
        max_tokens=1024,
        top_p=0.9,
        stream=True,
        stop=None,
    )

    response = ""
    for chunk in completion:
        response_content = chunk.choices[0].delta.content or ""
        response += response_content

    # print(response)

    with open('arabic_output.txt', 'w', encoding='utf-8') as file:
        # Write the Arabic text to the file
        file.write(response)
    return response, similar_docs


In [ ]:
query = "ما هو دور مسؤول البحث والتطوير؟"

query_data(query)

# Gradio

In [ ]:
!pip install beautifulsoup4 gradio==3.48.0 lxml

In [10]:
import gradio as gr
from gradio.themes.base import Base

with gr.Blocks(theme=Base(), title="Ma3aN Question Answer") as demo:
    # Add a logo
    gr.Image(value="/content/drive/MyDrive/Ma3an_QA/LOGO.png", width=200)

    gr.Markdown(
        """
        # Ma3aN Question Answer App using Vector DB + RAG Architecture
        """)
    # Textbox with a default question
    textbox = gr.Textbox(value="ما هو دور لجنه الميديا؟", label="Enter your Question:")

    with gr.Row():
        button = gr.Button("Submit", variant="primary")

    with gr.Column():
        output1 = gr.Textbox(lines=1, max_lines=10, label="Output with just Atlas Vector Search (returns text field as is):")
        output2 = gr.Textbox(lines=1, max_lines=10, label="Output generated by chaining Atlas Vector Search to Langchain's RetrieverQA + LLM:")

    # Call query_data function upon clicking the Submit button
    button.click(query_data, textbox, outputs=[output1, output2])

demo.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
IMPORTANT: You are using gradio version 3.48.0, however version 4.29.0 is available, please upgrade.
--------
Running on public URL: https://892fa3071e9d786700.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
